In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df=pd.read_csv("BANGALORE.csv")
df.head()

,_id,Date Time,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Skills and Perks
0,5f9fc1e3fb43357b9094a297,2020-11-02 13:52:57,Fashion Design,Enamor,Bangalore,Immediately,5000 /month,2 Months,11 Nov' 20,Internship,"[""MS-Office"",""Adobe Illustrator"",""CorelDRAW"",""..."
1,5f9fc1e4fb43357b9094a298,2020-11-02 13:52:59,Operations,Lakshmi Tools & Components,Hoskote,Immediately,12000 /month,1 Month,16 Nov' 20,Internship,"[""Chemistry"",""Certificate"","" Letter of recomme..."
2,5f9fc1e4fb43357b9094a299,2020-11-02 13:53:00,Photography,Evineon Technologies,Bangalore,Immediately,10000-12000 /month,3 Months,16 Nov' 20,Internship,"[""Flexible work hours"","" Free snacks & beverag..."
3,5f9fc1e5fb43357b9094a29a,2020-11-02 13:53:00,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,7000 /month,6 Months,16 Nov' 20,Internship,"[""MS-Office"",""Certificate"","" Letter of recomme..."
4,5f9fc1e6fb43357b9094a29b,2020-11-02 13:53:01,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,5000 /month,6 Months,16 Nov' 20,Internship,[]


In [3]:
for i in df.columns:
    print(i,":",df[i].nunique())

_id : 513
Date Time : 403
profile : 153
company : 378
Location : 21
Start Date : 2
Stipend : 98
Duration : 11
Apply by Date : 24
Offer : 2
Skills and Perks : 387


In [4]:
df.isna().sum()

_id                 0
Date Time           0
profile             0
company             0
Location            0
Start Date          0
Stipend             0
Duration            0
Apply by Date       0
Offer               0
Skills and Perks    0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df['Skills and Perks'].value_counts().head(10)

Skills and Perks
[]                                                                                           21
["Certificate"]                                                                              17
["Certificate"," Letter of recommendation"]                                                  14
["Certificate"," Letter of recommendation"," Flexible work hours"," Job offer"]              10
["Certificate"," Letter of recommendation"," Flexible work hours"," Informal dress code"]     9
["Certificate"," Letter of recommendation"," Flexible work hours"]                            8
["Certificate"," Job offer"]                                                                  6
["Certificate"," Letter of recommendation"," Job offer"]                                      5
["Certificate"," Letter of recommendation"," Informal dress code"]                            4
["Certificate"," Flexible work hours"]                                                        4
Name: count, dtype: int

In [7]:
df['Skills and Perks'].head(30)

0     ["MS-Office","Adobe Illustrator","CorelDRAW","...
1     ["Chemistry","Certificate"," Letter of recomme...
2     ["Flexible work hours"," Free snacks & beverag...
3     ["MS-Office","Certificate"," Letter of recomme...
4                                                    []
5     ["Digital Marketing","Certificate"," Letter of...
6     ["MS-PowerPoint","English Proficiency (Spoken)...
7     ["Digital Marketing","Certificate"," Letter of...
8     ["Social Media Marketing","Adobe Photoshop","V...
9     ["Python","Linux","Ethical Hacking","C Program...
10                         ["Salesforce","Certificate"]
11    ["Tally","MS-Excel","English Proficiency (Spok...
12    ["SQL","Hibernate (Java)","Spring MVC","REST A...
13    ["Python","Algorithms","Data Structures","Flas...
14    ["Adobe Photoshop","Adobe Illustrator","Adobe ...
15    ["Search Engine Optimization (SEO)","Certifica...
16    ["Certificate"," Letter of recommendation"," I...
17    ["Recruitment","English Proficiency (Spoke

In [8]:
skills = [".NET",
    "3ds Max",
    "ASP.NET",
    "Accounting",
    "Adobe After Effects",
    "Adobe Audition",
    "Adobe Creative Suite",
    "Adobe Dreamweaver",
    "Adobe Fireworks",
    "Adobe Flash",
    "Adobe Illustrator",
    "Adobe Indesign",
    "Adobe Photoshop",
    "Adobe Photoshop Lightroom CC",
    "Adobe Premiere Pro",
    "Adobe XD",
    "Algorithms",
    "Amazon Web Server (AWS)",
    "Amazon Web Services (AWS)",
    "Android",
    "Angular 2.0",
    "Angular 7.0",
    "AngularJS",
    "Animation",
    "Arduino",
    "Artifical Intelligence",
    "AutoCAD",
    "Autodesk Maya",
    "Autodesk Revit",
    "BIM (Building Information Modeling)",
    "Bengali Proficiency (Spoken)",
    "Bengali Proficiency (Written)",
    "Biology",
    "Blender 3D",
    "Blogging",
    "Bootstrap",
    "C Programming",
    "C++ Programming",
    "CSS",
    "Chemistry",
    "Client Relationship",
    "Cloud Computing",
    "CodeIgniter",
    "Computer Networking",
    "Computer Networks",
    "Copywriting",
    "CorelDRAW",
    "Creative Writing",
    "CRM",
    "CSS3",
    "Camtasia",
    "Canva",
    "Catia",
    "ChemDraw",
    "Cinema 4D",
    "Civil Engineering",
    "Communication Skills",
    "Content Writing",
    "Creative Designing",
    "Cryptography",
    "Data Entry",
    "Data Mining",
    "Data Science",
    "Data Structures",
    "Database Management System (DBMS)",
    "Deep Learning",
    "Design Thinking",
    "Digital Marketing",
    "Django",
    "Docker",
    "Drawing",
    "Economics",
    "Editing",
    "Electronics",
    "Embedded Systems",
    "Engineering Drawing",
    "English Proficiency (Spoken)",
    "English Proficiency (Written)",
    "Entrepreneurship",
    "Event Management",
    "Excel",
    "Fashion Designing",
    "Final Cut Pro",
    "Finance",
    "Flask",
    "French Proficiency (Spoken)",
    "French Proficiency (Written)",
    "Front-end Development",
    "Fusion 360",
    "Git",
    "GitHub",
    "Google Analytics",
    "Google Cloud Computing",
    "Google SketchUp",
    "Hindi Proficiency (Spoken)",
    "Hindi Proficiency (Written)",
    "HTML",
    "Illustration",
    "Internet of Things (IoT)",
    "Interview Skills",
    "Java",
    "JavaScript",
    "Joomla",
    "Journalism",
    "Kotlin",
    "Laravel",
    "Leadership",
    "Linux",
    "Machine Learning",
    "Management",
    "Marketing",
    "MATLAB",
    "Mechanical Engineering",
    "Microsoft Excel",
    "Microsoft PowerPoint",
    "Microsoft Word",
    "MongoDB",
    "MS-Excel",
    "MS-Office",
    "MS-PowerPoint",
    "MS-Word",
    "MySQL",
    "Natural Language Processing (NLP)",
    "Negotiation",
    "Node.js",
    "Objective C",
    "OpenCV",
    "Operating System",
    "Oracle",
    "PCB Design",
    "Perl",
    "Photography",
    "PHP",
    "PostgreSQL",
    "Problem Solving",
    "Programming",
    "Project Management",
    "Prototyping",
    "Public Speaking",
    "Python",
    "R Programming",
    "ReactJS",
    "Recruitment",
    "Research and Analytics",
    "REST API",
    "Robotics",
    "Ruby on Rails",
    "SAP",
    "Scala",
    "Search Engine Optimization (SEO)",
    "Social Media Marketing",
    "Software Testing",
    "SolidWorks",
    "Spanish Proficiency (Spoken)",
    "Spanish Proficiency (Written)",
    "Spring MVC",
    "SQL",
    "Statistics",
    "Swift",
    "Tableau",
    "Tally",
    "Team Management",
    "Technical Writing",
    "Time Management",
    "Unity 3D",
    "Unreal Engine",
    "Video Editing",
    "Virtual Reality (VR)",
    "Vue Js",
    "Web Development",
    "WordPress",
    "Writing"
]


perks = ['Certificate','Letter of recommendation','Flexible work hours','Free snacks & beverages','5 days a week','Job offer']

In [9]:
import ast

def to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return [x]
    return x

df['Skills and Perks'] = df['Skills and Perks'].apply(to_list)


In [10]:
def split_skills_perks(items):
    skills_col, perks_col = [], []
    for item in items:
        item_clean = item.strip()
        if item_clean in perks:
            perks_col.append(item_clean)
        else:
            skills_col.append(item_clean)
    return pd.Series([', '.join(skills_col), ', '.join(perks_col)])

df[['Skills', 'Perks']] = df['Skills and Perks'].apply(split_skills_perks)

In [11]:
df=df.drop(columns='Skills and Perks')
df.head()

,_id,Date Time,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Skills,Perks
0,5f9fc1e3fb43357b9094a297,2020-11-02 13:52:57,Fashion Design,Enamor,Bangalore,Immediately,5000 /month,2 Months,11 Nov' 20,Internship,"MS-Office, Adobe Illustrator, CorelDRAW",Certificate
1,5f9fc1e4fb43357b9094a298,2020-11-02 13:52:59,Operations,Lakshmi Tools & Components,Hoskote,Immediately,12000 /month,1 Month,16 Nov' 20,Internship,"Chemistry, Informal dress code","Certificate, Letter of recommendation, Free sn..."
2,5f9fc1e4fb43357b9094a299,2020-11-02 13:53:00,Photography,Evineon Technologies,Bangalore,Immediately,10000-12000 /month,3 Months,16 Nov' 20,Internship,,"Flexible work hours, Free snacks & beverages"
3,5f9fc1e5fb43357b9094a29a,2020-11-02 13:53:00,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,7000 /month,6 Months,16 Nov' 20,Internship,"MS-Office, Informal dress code","Certificate, Letter of recommendation, Flexibl..."
4,5f9fc1e6fb43357b9094a29b,2020-11-02 13:53:01,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,5000 /month,6 Months,16 Nov' 20,Internship,,


In [12]:
df['Skills'].replace("", "None", inplace=True)
df['Perks'].replace("", "None", inplace=True)
df.head()

C:\Users\boomi\AppData\Local\Temp\ipykernel_18084\3340754277.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Skills'].replace("", "None", inplace=True)
C:\Users\boomi\AppData\Local\Temp\ipykernel_18084\3340754277.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,_id,Date Time,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Skills,Perks
0,5f9fc1e3fb43357b9094a297,2020-11-02 13:52:57,Fashion Design,Enamor,Bangalore,Immediately,5000 /month,2 Months,11 Nov' 20,Internship,"MS-Office, Adobe Illustrator, CorelDRAW",Certificate
1,5f9fc1e4fb43357b9094a298,2020-11-02 13:52:59,Operations,Lakshmi Tools & Components,Hoskote,Immediately,12000 /month,1 Month,16 Nov' 20,Internship,"Chemistry, Informal dress code","Certificate, Letter of recommendation, Free sn..."
2,5f9fc1e4fb43357b9094a299,2020-11-02 13:53:00,Photography,Evineon Technologies,Bangalore,Immediately,10000-12000 /month,3 Months,16 Nov' 20,Internship,None,"Flexible work hours, Free snacks & beverages"
3,5f9fc1e5fb43357b9094a29a,2020-11-02 13:53:00,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,7000 /month,6 Months,16 Nov' 20,Internship,"MS-Office, Informal dress code","Certificate, Letter of recommendation, Flexibl..."
4,5f9fc1e6fb43357b9094a29b,2020-11-02 13:53:01,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,5000 /month,6 Months,16 Nov' 20,Internship,None,None


In [13]:
df['Skills_list'] = df['Skills'].apply(lambda x: x.split(", ") if x != "None" else [])

# Create MultiLabelBinarizer for all skills
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
skills_encoded = pd.DataFrame(mlb.fit_transform(df['Skills_list']), columns=mlb.classes_)

# Keep only top skills appearing more than 5 times
top_skills = skills_encoded.sum()[skills_encoded.sum() > 5].index
skills_encoded_top = skills_encoded[top_skills]

# Add skills count column
df['Skills_count'] = df['Skills_list'].apply(len)

# Concatenate top skills
df = pd.concat([df, skills_encoded_top], axis=1)

In [14]:
df["Date Time"]=pd.to_datetime(df["Date Time"],errors="coerce")
df["Date"]=df["Date Time"].dt.day
df["month"]=df["Date Time"].dt.month
df["year"]=df["Date Time"].dt.year
df["time"]=df["Date Time"].dt.time
df=df.drop(columns='Date Time')
df.head()

,_id,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Skills,...,Social Media Marketing,Tally,Video Editing,Video Making,WordPress,iOS,Date,month,year,time
0,5f9fc1e3fb43357b9094a297,Fashion Design,Enamor,Bangalore,Immediately,5000 /month,2 Months,11 Nov' 20,Internship,"MS-Office, Adobe Illustrator, CorelDRAW",...,0,0,0,0,0,0,2,11,2020,13:52:57
1,5f9fc1e4fb43357b9094a298,Operations,Lakshmi Tools & Components,Hoskote,Immediately,12000 /month,1 Month,16 Nov' 20,Internship,"Chemistry, Informal dress code",...,0,0,0,0,0,0,2,11,2020,13:52:59
2,5f9fc1e4fb43357b9094a299,Photography,Evineon Technologies,Bangalore,Immediately,10000-12000 /month,3 Months,16 Nov' 20,Internship,None,...,0,0,0,0,0,0,2,11,2020,13:53:00
3,5f9fc1e5fb43357b9094a29a,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,7000 /month,6 Months,16 Nov' 20,Internship,"MS-Office, Informal dress code",...,0,0,0,0,0,0,2,11,2020,13:53:00
4,5f9fc1e6fb43357b9094a29b,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,5000 /month,6 Months,16 Nov' 20,Internship,None,...,0,0,0,0,0,0,2,11,2020,13:53:01


In [15]:
df=df.drop(columns="_id")

In [16]:
df.columns

Index(['profile', 'company', 'Location', 'Start Date', 'Stipend', 'Duration',
       'Apply by Date', 'Offer', 'Skills', 'Perks', 'Skills_list',
       'Skills_count', '3ds Max', 'Accounting', 'Adobe After Effects',
       'Adobe Creative Suite', 'Adobe Illustrator', 'Adobe Photoshop',
       'Adobe Photoshop Lightroom CC', 'Adobe Premiere Pro', 'Android',
       'AutoCAD', 'Blogging', 'Bootstrap', 'CSS', 'Client Relationship',
       'Copywriting', 'CorelDRAW', 'Creative Writing', 'Digital Marketing',
       'Email Marketing', 'English Proficiency (Spoken)',
       'English Proficiency (Written)', 'Facebook Marketing', 'Flutter',
       'Google AdWords', 'Google Analytics', 'HTML',
       'Hindi Proficiency (Spoken)', 'Informal dress code',
       'Instagram Marketing', 'Java', 'JavaScript',
       'Kannada Proficiency (Spoken)', 'Linux', 'MS-Excel', 'MS-Office',
       'MS-PowerPoint', 'MS-Word', 'Node.js', 'PHP', 'Photography', 'Python',
       'ReactJS', 'SQL', 'Salesforce', 'Searc

In [17]:
df['Stipend'].unique()

array([' 5000 /month', ' 12000 /month', ' 10000-12000 /month',
       ' 7000 /month', ' 5000-10000 /month', ' 2000-3000 /month',
       ' 4000-7000 /month', ' 8000 /month', ' 15000-18000 /month',
       ' 3000 /month', ' 8000-12000 /month', ' 9000-12000 /month',
       ' 10000-20000 /month', ' 7000-13000 /month', ' 4000 /month',
       ' 7000-12000 /month', ' 10000 /month', ' 15000 /month',
       ' 2000 /month +  Incentives', ' 3500 /month', ' 3000-5000 /month',
       ' 2000 /month', ' 11000 /month', ' 20000 /month',
       ' 10000-15000 /month', ' 8000-10000 /month', ' 2000 lump sum',
       ' 15000-25000 /month', ' 18000 /month', ' 4000-9000 /month',
       ' 8000-15000 /month', ' 7000-10000 /month', ' 6000-7000 /month',
       ' 25000 /month', ' 8000-13000 /month',
       ' 7000 /month +  Incentives', ' 9500 /month',
       ' 12000 /month +  Incentives', ' 17500 /month',
       ' 5000-7000 /month', 'Performance Based', ' 15000-20000 /month',
       ' 7500 /month', ' 10000-14000 /m

In [18]:
import re

def preprocess_stipend(x):
    x = str(x).strip()

    # Unpaid
    if "Unpaid" in x:
        return pd.Series([0, 0, 0, "none", 1, 0, 0])

    # Performance Based
    if "Performance" in x:
        return pd.Series([0, 0, 0, "none", 0, 1, 0])

    # Detect Incentives
    incentives = 1 if "Incentives" in x else 0

    # Type (month, week, lump sum, etc.)
    if "month" in x:
        stype = "month"
    elif "week" in x:
        stype = "week"
    elif "lump sum" in x:
        stype = "lump_sum"
    else:
        stype = "other"

    # Extract numbers
    numbers = re.findall(r'\d+', x)
    numbers = list(map(int, numbers))

    if len(numbers) == 0:
        return pd.Series([0, 0, 0, stype, 0, 0, incentives])
    elif len(numbers) == 1:
        return pd.Series([numbers[0], numbers[0], numbers[0], stype, 0, 0, incentives])
    elif len(numbers) == 2:
        avg = np.mean(numbers)
        return pd.Series([numbers[0], numbers[1], avg, stype, 0, 0, incentives])
    else:
        return pd.Series([0, 0, 0, stype, 0, 0, incentives])

# Apply to dataset
df[['stipend_min','stipend_max','stipend_avg','stipend_type',
    'stipend_unpaid','stipend_perf','stipend_incentives']] = df['Stipend'].apply(preprocess_stipend)
df=df.drop(columns='Stipend')


In [19]:
df['Duration'].unique()

array(['2 Months', '1 Month', '3 Months', '6 Months', '4 Weeks',
       '4 Months', '5 Months', '1 Week', '6 Weeks', '2 Weeks',
       '12 Months'], dtype=object)

In [20]:
def preprocess_duration(x):
    x = str(x).strip().lower()

    # Extract number
    num = re.findall(r'\d+', x)
    if not num:
        return 0
    num = int(num[0])

    # Convert to months
    if "month" in x:
        months = num
    elif "week" in x:
        months = num / 4   # approx 4 weeks = 1 month
    else:
        months = 0

    return months

# Apply
df['duration_months'] = df['Duration'].apply(preprocess_duration)
df=df.drop(columns='Duration')
df=df.drop(columns=['Skills','Skills_list'])
df.head()

,profile,company,Location,Start Date,Apply by Date,Offer,Perks,Skills_count,3ds Max,Accounting,...,year,time,stipend_min,stipend_max,stipend_avg,stipend_type,stipend_unpaid,stipend_perf,stipend_incentives,duration_months
0,Fashion Design,Enamor,Bangalore,Immediately,11 Nov' 20,Internship,Certificate,3,0,0,...,2020,13:52:57,5000,5000,5000.0,month,0,0,0,2.0
1,Operations,Lakshmi Tools & Components,Hoskote,Immediately,16 Nov' 20,Internship,"Certificate, Letter of recommendation, Free sn...",2,0,0,...,2020,13:52:59,12000,12000,12000.0,month,0,0,0,1.0
2,Photography,Evineon Technologies,Bangalore,Immediately,16 Nov' 20,Internship,"Flexible work hours, Free snacks & beverages",0,0,0,...,2020,13:53:00,10000,12000,11000.0,month,0,0,0,3.0
3,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,16 Nov' 20,Internship,"Certificate, Letter of recommendation, Flexibl...",2,0,0,...,2020,13:53:00,7000,7000,7000.0,month,0,0,0,6.0
4,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,16 Nov' 20,Internship,None,0,0,0,...,2020,13:53:01,5000,5000,5000.0,month,0,0,0,6.0


In [21]:
df['Offer'].unique()

array(['Internship', 'Internship with job offer'], dtype=object)

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Offer_encoded'] = le.fit_transform(df['Offer'])
df=df.drop(columns='Offer')
df.head()

,profile,company,Location,Start Date,Apply by Date,Perks,Skills_count,3ds Max,Accounting,Adobe After Effects,...,time,stipend_min,stipend_max,stipend_avg,stipend_type,stipend_unpaid,stipend_perf,stipend_incentives,duration_months,Offer_encoded
0,Fashion Design,Enamor,Bangalore,Immediately,11 Nov' 20,Certificate,3,0,0,0,...,13:52:57,5000,5000,5000.0,month,0,0,0,2.0,0
1,Operations,Lakshmi Tools & Components,Hoskote,Immediately,16 Nov' 20,"Certificate, Letter of recommendation, Free sn...",2,0,0,0,...,13:52:59,12000,12000,12000.0,month,0,0,0,1.0,0
2,Photography,Evineon Technologies,Bangalore,Immediately,16 Nov' 20,"Flexible work hours, Free snacks & beverages",0,0,0,0,...,13:53:00,10000,12000,11000.0,month,0,0,0,3.0,0
3,Social Media Marketing,VisualNet Private Limited,Bangalore,Immediately,16 Nov' 20,"Certificate, Letter of recommendation, Flexibl...",2,0,0,0,...,13:53:00,7000,7000,7000.0,month,0,0,0,6.0,0
4,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,Immediately,16 Nov' 20,None,0,0,0,0,...,13:53:01,5000,5000,5000.0,month,0,0,0,6.0,0


In [23]:
df['Start Date'].unique()

array(['Immediately', 'Nothing'], dtype=object)

In [24]:
le = LabelEncoder()
df['StartDate_encoded'] = le.fit_transform(df['Start Date'])
df=df.drop(columns='Start Date')
df.head()

,profile,company,Location,Apply by Date,Perks,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,...,stipend_min,stipend_max,stipend_avg,stipend_type,stipend_unpaid,stipend_perf,stipend_incentives,duration_months,Offer_encoded,StartDate_encoded
0,Fashion Design,Enamor,Bangalore,11 Nov' 20,Certificate,3,0,0,0,0,...,5000,5000,5000.0,month,0,0,0,2.0,0,0
1,Operations,Lakshmi Tools & Components,Hoskote,16 Nov' 20,"Certificate, Letter of recommendation, Free sn...",2,0,0,0,0,...,12000,12000,12000.0,month,0,0,0,1.0,0,0
2,Photography,Evineon Technologies,Bangalore,16 Nov' 20,"Flexible work hours, Free snacks & beverages",0,0,0,0,0,...,10000,12000,11000.0,month,0,0,0,3.0,0,0
3,Social Media Marketing,VisualNet Private Limited,Bangalore,16 Nov' 20,"Certificate, Letter of recommendation, Flexibl...",2,0,0,0,0,...,7000,7000,7000.0,month,0,0,0,6.0,0,0
4,Business Development (Sales),DM Consulting (India) Private Limited,Bangalore,16 Nov' 20,None,0,0,0,0,0,...,5000,5000,5000.0,month,0,0,0,6.0,0,0


In [25]:
df['Location'].unique()

array(['Bangalore', 'Hoskote', 'Coimbatore', 'Chandigarh', 'Delhi',
       'Agra', 'Chennai', 'Ahmedabad', 'Pune', 'Secunderabad', 'Kolhapur',
       'Kolkata', 'Mangaluru', 'Mumbai', 'Hubli', 'Gurgaon',
       'Thiruvananthapuram', 'Thrissur', 'Visakhapatnam', 'Faridabad',
       'Indore'], dtype=object)

In [26]:
le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])
df=df.drop(columns='Location')
df.head()

,profile,company,Apply by Date,Perks,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,Adobe Illustrator,...,stipend_max,stipend_avg,stipend_type,stipend_unpaid,stipend_perf,stipend_incentives,duration_months,Offer_encoded,StartDate_encoded,Location_encoded
0,Fashion Design,Enamor,11 Nov' 20,Certificate,3,0,0,0,0,1,...,5000,5000.0,month,0,0,0,2.0,0,0,2
1,Operations,Lakshmi Tools & Components,16 Nov' 20,"Certificate, Letter of recommendation, Free sn...",2,0,0,0,0,0,...,12000,12000.0,month,0,0,0,1.0,0,0,9
2,Photography,Evineon Technologies,16 Nov' 20,"Flexible work hours, Free snacks & beverages",0,0,0,0,0,0,...,12000,11000.0,month,0,0,0,3.0,0,0,2
3,Social Media Marketing,VisualNet Private Limited,16 Nov' 20,"Certificate, Letter of recommendation, Flexibl...",2,0,0,0,0,0,...,7000,7000.0,month,0,0,0,6.0,0,0,2
4,Business Development (Sales),DM Consulting (India) Private Limited,16 Nov' 20,None,0,0,0,0,0,0,...,5000,5000.0,month,0,0,0,6.0,0,0,2


In [27]:
df['company'].unique()

array(['Enamor', 'Lakshmi Tools & Components', 'Evineon Technologies',
       'VisualNet Private Limited',
       'DM Consulting (India) Private Limited',
       'Zyena Technologies SDN, BHD, Malaysia',
       'Visual Reality Events & Production', 'Webi7 Digital Media',
       'StoreEDge Solutions Private Limited',
       'Resec Systems Private Limited', 'Transchamp',
       'Gayathri Design Studio', 'TesQuirel Solutions Private Limited',
       'Knedge', 'Bodhsara Wellness And Salt Studio',
       'Brainstorm Consulting', 'Inovexic', 'Cloud Incorporated',
       'OAP India Private Limited', 'Prashant  Dayama', 'Gloify',
       'INEL Power System Engineers',
       'Skillate Laboratories Private Limited', 'United Business',
       'Craftech 360', 'UrbanServ',
       'Mech Skillz Engineering Solutions Private Limited',
       'GT Informatics', 'Yes Property Ventures Private Limited',
       'Rolla Online Institute', 'Digi Streaks',
       'Thirumathi Herbal & Natural Products',
       '

In [28]:
le = LabelEncoder()
df['Company_encoded'] = le.fit_transform(df['company'])
df=df.drop(columns='company')
df.head(2)

,profile,Apply by Date,Perks,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,Adobe Illustrator,Adobe Photoshop,...,stipend_avg,stipend_type,stipend_unpaid,stipend_perf,stipend_incentives,duration_months,Offer_encoded,StartDate_encoded,Location_encoded,Company_encoded
0,Fashion Design,11 Nov' 20,Certificate,3,0,0,0,0,1,0,...,5000.0,month,0,0,0,2.0,0,0,2,99
1,Operations,16 Nov' 20,"Certificate, Letter of recommendation, Free sn...",2,0,0,0,0,0,0,...,12000.0,month,0,0,0,1.0,0,0,9,193


In [29]:
df['Perks'].unique()

array(['Certificate',
       'Certificate, Letter of recommendation, Free snacks & beverages',
       'Flexible work hours, Free snacks & beverages',
       'Certificate, Letter of recommendation, Flexible work hours, 5 days a week, Free snacks & beverages',
       'None',
       'Certificate, Letter of recommendation, Flexible work hours, 5 days a week, Free snacks & beverages, Job offer',
       'Certificate, Letter of recommendation, Job offer',
       'Certificate, Free snacks & beverages',
       'Certificate, Letter of recommendation, Flexible work hours, 5 days a week',
       'Certificate, 5 days a week',
       'Certificate, Letter of recommendation, 5 days a week',
       'Certificate, Letter of recommendation, 5 days a week, Job offer',
       'Certificate, Letter of recommendation, 5 days a week, Free snacks & beverages, Job offer',
       'Certificate, Letter of recommendation, Flexible work hours, Job offer',
       'Certificate, Letter of recommendation, Flexible work ho

In [30]:
# Split each string by comma and strip spaces
all_perks = set()
for perks in df['Perks']:
    if perks != 'None':
        for perk in perks.split(','):
            all_perks.add(perk.strip())

for perk in all_perks:
    df[f'Perk_{perk}'] = df['Perks'].apply(lambda x: int(perk in x) if x != 'None' else 0)

In [31]:
df=df.drop(columns='Perks')
print("Shape:",df.shape)

Shape: (513, 77)


In [32]:
df.head(2)

,profile,Apply by Date,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,Adobe Illustrator,Adobe Photoshop,Adobe Photoshop Lightroom CC,...,Offer_encoded,StartDate_encoded,Location_encoded,Company_encoded,Perk_Letter of recommendation,Perk_Certificate,Perk_5 days a week,Perk_Free snacks & beverages,Perk_Job offer,Perk_Flexible work hours
0,Fashion Design,11 Nov' 20,3,0,0,0,0,1,0,0,...,0,0,2,99,0,1,0,0,0,0
1,Operations,16 Nov' 20,2,0,0,0,0,0,0,0,...,0,0,9,193,1,1,0,1,0,0


In [33]:
df['profile'].unique()

array(['Fashion Design', 'Operations', 'Photography',
       'Social Media Marketing', 'Business Development (Sales)',
       'Event Management/Wedding Planning', 'Digital Marketing',
       'Graphic Design', 'Cyber Security',
       'Java App Development (Spring Boot & Hibernate)',
       'Python Web Application', 'Search Engine Optimization (SEO)',
       'Human Resources (HR)', 'Law/ Legal', 'iOS App Development',
       'Customer Support', 'Firmware Development', 'Marketing',
       'Online Marketing', 'Career Returnee Program (Women)',
       'Financial Advisor Training', 'Civil Engineering',
       'Interior Design',
       'Subject Expert For Creating Tutorials (Python)',
       'Web Development', 'Acting & Cinematography', 'Telecalling',
       'Research And Development (Grey Water Treatment)',
       'Inventory Management', 'Assembly & Manufacturing',
       'Event Management', 'Customer Service', 'Content Writing',
       'Big Data Development And Testing', 'Finance', 'Counse

In [34]:
le = LabelEncoder()
df['Profile_encoded'] = le.fit_transform(df['profile'])
df=df.drop(columns='profile')
df.head(2)

,Apply by Date,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,Adobe Illustrator,Adobe Photoshop,Adobe Photoshop Lightroom CC,Adobe Premiere Pro,...,StartDate_encoded,Location_encoded,Company_encoded,Perk_Letter of recommendation,Perk_Certificate,Perk_5 days a week,Perk_Free snacks & beverages,Perk_Job offer,Perk_Flexible work hours,Profile_encoded
0,11 Nov' 20,3,0,0,0,0,1,0,0,0,...,0,2,99,0,1,0,0,0,0,55
1,16 Nov' 20,2,0,0,0,0,0,0,0,0,...,0,9,193,1,1,0,1,0,0,99


In [35]:
df=df.drop(columns='Apply by Date')
df.head()

,Skills_count,3ds Max,Accounting,Adobe After Effects,Adobe Creative Suite,Adobe Illustrator,Adobe Photoshop,Adobe Photoshop Lightroom CC,Adobe Premiere Pro,Android,...,StartDate_encoded,Location_encoded,Company_encoded,Perk_Letter of recommendation,Perk_Certificate,Perk_5 days a week,Perk_Free snacks & beverages,Perk_Job offer,Perk_Flexible work hours,Profile_encoded
0,3,0,0,0,0,1,0,0,0,0,...,0,2,99,0,1,0,0,0,0,55
1,2,0,0,0,0,0,0,0,0,0,...,0,9,193,1,1,0,1,0,0,99
2,0,0,0,0,0,0,0,0,0,0,...,0,2,104,0,0,0,1,0,1,106
3,2,0,0,0,0,0,0,0,0,0,...,0,2,346,1,1,1,1,0,1,126
4,0,0,0,0,0,0,0,0,0,0,...,0,2,70,0,0,0,0,0,0,18


In [36]:
df.to_csv('cleaned_dataset.csv')
print("Saved successfully")

Saved successfully
